## JSON DB

In [10]:
import json
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from tqdm import tqdm
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [9]:
llm = ChatOpenAI(model="gpt-4", temperature=0)  # Используем GPT-4 для точности

# Функция для приведения текста к нижнему регистру и удаления лишних пробелов
def clean_text(text):
    return " ".join(text.lower().split())

# Функция для генерации нового JSON-формата через LLM
def transform_product(product):
    prompt = f"""
    Ты AI-помощник, который помогает преобразовать JSON-данные о товарах.
    Тебе дан товар в формате JSON, и тебе нужно вернуть его в новой структуре.
    
    **Исходный формат:**
    {{
        "name": "Название товара",
        "description": "Описание товара",
        "sizes": ["размеры"],
        "stock": [количество],
        "price": цена
    }}

    **Новая структура:**
    {{
        "id": "уникальный id (используй 5 случайных цифр)",
        "text": "name" + "description",
        "metadata": {{
            "category": "Костюм / Куртка / Брюки / Обувь / Комплементарное (определи по названию и описанию, указывай Комплементарное если товар не является костюмом, курткой, брюками или обувью, не придумывай новые категории, указывай только из этого списка)",
            "season": "Зима / Лето / Демисезон (определи из описания)",
            "gender": "Мужской / Женский / Унисекс (если нет упоминаний пола, ставь унисекс)",
            "material":  "["список материалов (определи из описания)"]", //список переведи в строку без указания отсутопов или новой строки
            "sizes": "["размеры"]", // сделай строку из листа не изменяя сами данные без указания отсутопов или новой строки
            "stock": "[количество]", // сделай строку из листа не изменяя сами данные без указания отсутопов или новой строки
            "price": цена,
            "currency": "RUB"
            
        }}
    }}
    
    В ответе должен быть ТОЛЬКО JSON, без объяснений.
    
    Вот товар, который нужно обработать:
    {json.dumps(product, ensure_ascii=False)}
    """
    
    response = llm.invoke([SystemMessage(content="Ты AI, который преобразует данные товаров."), HumanMessage(content=prompt)])
    
    try:
        new_product = json.loads(response.content)  # Декодируем JSON
        new_product["text"] = clean_text(new_product["text"])  # Приведение к нижнему регистру
        return new_product
    except json.JSONDecodeError:
        print(f"Ошибка обработки JSON для товара: {product['name']}")
        return None




/tmp/ipykernel_89317/3757979702.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)  # Используем GPT-4 для точности


In [12]:
files = os.listdir("datasets_json/json_price_suplier")

for file in files:
    file_name = f"datasets_json/json_price_suplier/{file}"
    output_file = f"datasets_json/json_price_full/{file}"

    # Читаем исходный JSON-файл
    with open(file_name, "r", encoding="utf-8") as f:
        products = json.load(f)

    # Преобразуем товары
    transformed_products = []
    for product in tqdm(products, desc="Обработка товаров"):
        new_product = transform_product(product)
        if new_product:
            transformed_products.append(new_product)

    # Сохраняем результат
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(transformed_products, f, ensure_ascii=False, indent=4)

    print("✅ Готово! Данные сохранены в transformed_products.json")

Обработка товаров: 100%|██████████| 22/22 [02:41<00:00,  7.33s/it]


✅ Готово! Данные сохранены в transformed_products.json


Обработка товаров: 100%|██████████| 99/99 [11:04<00:00,  6.71s/it]


✅ Готово! Данные сохранены в transformed_products.json


Обработка товаров: 100%|██████████| 40/40 [04:13<00:00,  6.35s/it]


✅ Готово! Данные сохранены в transformed_products.json


Обработка товаров: 100%|██████████| 12/12 [03:07<00:00, 15.61s/it]


✅ Готово! Данные сохранены в transformed_products.json


Обработка товаров:   0%|          | 0/37 [00:02<?, ?it/s]


KeyboardInterrupt: 

## JSON Agent client data

In [2]:
from dotenv import load_dotenv
import os
import json
import openai
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_community.document_loaders import UnstructuredFileLoader

load_dotenv()


True

### prompt templates

#### обработка заявки

prompt = f"""
        Ты получишь текстовые данные с описанием товаров. Они могут быть как табличными так и текст с описанием товаров.  
        Верни два списка со следующими структурами. 
        1. Общая характеристика заявки со струткурой:
        [
        {{
            "номер заявки": int,         // используй UUID v4 для генерации уникального номера заявки
            "дата": datetime,  // дата прихода заявки
            "товары": [int]        // уникальные id для каждого товара в этой заявке 
        }}
        ]
        2.
        [
        {{
            "ID": int,         // id-товара из первого json файла
            "описание": "string" // описание товара, если нет оставь пустым 
            "наименование": "string",  // наименование товара
            "размеры": ["string/int"],      // Список доступных размеров (если нет, укажи []), размеры могут иметь следующий вид: l70-176;48-50;S/M/L
            "кол-во": ["int"]         // Количество товаров для каждого размера или общее количество (если размеров нет, укажи [общее число]), количество часто это число перед шт. : 5 шт.
        }}
        ]
        Правила обработки данных:
        Извлекай только нужные данные: игнорируй любые другие колонки и текст.
        "размеры" – если размеры указаны, представь их в виде списка ["S", "M", "L"] или ["44", "56"], если их нет — укажи ["one-size"].
        "кол-во" – Если есть размеры, указывай количество товаров для каждого размера в том же порядке, что и "размеры". 
        Если размеров нет, но указано общее количество, записывай его как список с одним числом ["количество"]. 
        
        Важно:
        Если есть таблицы: анализируй заголовки таблиц и извлекай данные корректно.
        Соблюдай указанную структуру без лишний слов. Ответ должен содержать два массива через запятую.

        Вот данные:
        {text}
    """

#### создание бд

    prompt = f"""
        Ты получишь текстовые данные с описанием товаров и их цен. Это таблица.  
        Верни json структуру:
        [
        {{
            "id": int,  # Генерация ID, уникального для каждого товара в таблице
            "name": string, # наименование товара, в тексте это представлено как коротное описание
            "description": string, # полное описание товара
            "price": float,  # Преобразуем в число # цена товара
            "currency": "руб." # одинаковая для всех товаров, не зависит от текста
        }}
        ]
       
        Правила обработки:
        - Верни данные в формате json
        - Придумай уникальный ID для каждого товара
        - Преобразуй цену в число
        - Преобразуй текст в нижний регистр
        - Удали лишние пробелы
        - Лемматизируй текст
        
        Важно:
        Анализируй заголовки таблиц и извлекай данные корректно.
        Соблюдай указанную структуру без лишний слов. Ответ должен содержать ТОЛЬКО json структуру.

        Вот данные:
        {text}
    """

### creating bd

In [3]:
def extract_text_from_file(file_path):
    text_data = []
    try:
        loader = UnstructuredFileLoader(file_path)
        docs = loader.load()
        text_data.append("\n\n".join([doc.page_content for doc in docs]))
    except Exception as e:
        print(f"⚠️ Ошибка при обработке {file_path}: {e}")

    return "\n\n".join(text_data)

def convert_to_json(text):
    prompt = f"""
        Ты получишь текстовые данные с описанием товаров и их цен. Это таблица.  
        Верни json структуру:
        [
        {{
            "name": "Название товара", // наименование товара
            "description": "Описание товара", // описание товара, если нет оставь пустым 
            "sizes": ["размеры"], // Список доступных размеров (если нет, укажи []), размеры могут иметь следующий вид: l70-176;48-50;S/M/L
            "stock": [количество], // Количество товаров для каждого размера или общее количество (если размеров нет, укажи [общее число]), количество часто это число перед шт. : 5 шт.
            "price": цена
        }}
        ]
       
        Правила обработки:
        - Верни данные в формате json
        - Преобразуй цену в число
        - Преобразуй текст в нижний регистр
        - Удали лишние пробелы
        
        Важно:
        Анализируй заголовки таблиц и извлекай данные корректно.
        Соблюдай указанную структуру без лишний слов. Ответ должен содержать ТОЛЬКО json структуру.

        Вот данные:
        {text}
    """
    
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    response = llm([HumanMessage(content=prompt)])
    return response.content.strip("```json").strip("```")

def process_files(dir):
    
    for file_name in os.listdir(dir):
        file_path = os.path.join(dir, file_name)
        text_data = extract_text_from_file(file_path)
        json_data = convert_to_json(text_data)
    
        #return json_data
        output_file = os.path.splitext(f'json_price/{file_name}')[0]+'.json'

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(json.loads(json_data), f, indent=4, ensure_ascii=False)
        print(f"✅ Данные сохранены в {output_file}!")

In [10]:
text_data = extract_text_from_file("exmp_data/ЗАЯВКА НА СПЕЦОДЕЖДУ (1).pdf")
json_data = convert_to_json(text_data)


In [ ]:
process_files("datasets_raw/прайсы_поставщиков")

## vecotr db

In [1]:
import json
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from uuid import uuid4
import os

# 2. Настраиваем ChromaDB
chroma_client = chromadb.PersistentClient(path="chroma_db_price")  # Хранит данные на диске
collection = chroma_client.get_or_create_collection(name="products")

# 3. Используем OpenAI для векторизации (замени на свою модель, если надо)
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

for file_name in os.listdir("datasets_json/json_price"):
    file_path = "datasets_json/json_price/" + file_name
    # 1. Загружаем JSON-файл
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 4. Преобразуем данные и загружаем в Chroma
    for item in data:
        vector = embedding_function.embed_query(item["text"])  # Векторизация текста
        collection.add(
            ids=str(uuid4()),
            embeddings=[vector],
            metadatas=[item["metadata"]],
            documents=[item["text"]]
        )

    print("Данные успешно загружены в ChromaDB!")


/tmp/ipykernel_266289/3198731672.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")


Данные успешно загружены в ChromaDB!
Данные успешно загружены в ChromaDB!
Данные успешно загружены в ChromaDB!
Данные успешно загружены в ChromaDB!
Данные успешно загружены в ChromaDB!
Данные успешно загружены в ChromaDB!


In [4]:
#from langchain_ollama import OllamaEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

from uuid import uuid4
import json
from langchain.schema import Document
import os

In [2]:
import re
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = text.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmatized_words)

In [5]:
docs = []

for file_name in os.listdir("json_price"):
    file_path = "json_price/" + file_name
    with open(file_path, "r", encoding="utf-8") as f:
        products = json.load(f)


    for product in products:
        name = preprocess_text(product["name"])
        description = preprocess_text(product["description"])
        metadata = {key: str(product[key]) for key in product if key not in ["name", "description"]}
        metadata["currency"] = "RUB"

        # Create a LangChain Document object
        doc = Document(page_content=f"{name}. {description}", metadata=metadata)
        docs.append(doc)

In [6]:
from langchain_chroma import Chroma


#model_name = "DeepPavlov/rubert-base-cased"
#embedding_model = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-base")
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large")

vector_store = Chroma(
    collection_name="products",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db_price",  # Where to save data locally, remove if not necessary
)

In [7]:
uuids = [str(uuid4()) for _ in range(len(docs))]

In [ ]:
vector_store.add_documents(documents=docs, ids = uuids)

## retriever

In [32]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(embedding_function=embedding_model, collection_name = "products", persist_directory="./chroma_db_price")

In [33]:
filters = {'$and': [{'category': {'$eq': "Комплементарное"}}, {'season': {'$eq': "Лето"}}, {'gender': {'$eq': 'Унисекс'}}]}
retriever = vector_store.as_retriever(search_kwargs={'filter':filters})

In [36]:
retriever.invoke("Футболка")

[Document(id='02972398-9cad-4423-a906-ae40af997148', metadata={'category': 'Комплементарное', 'currency': 'RUB', 'gender': 'Унисекс', 'material': '', 'price': 290, 'season': 'Лето', 'sizes': 'one-size', 'stock': '1'}, page_content='бейсболка васильковый'),
 Document(id='290d6438-4aab-4d77-9c80-899140e9c874', metadata={'category': 'Комплементарное', 'currency': 'RUB', 'gender': 'Унисекс', 'material': '[]', 'price': 500, 'season': 'Лето', 'sizes': '["one-size"]', 'stock': '[1]'}, page_content='рубашка-поло с коротким рукавом красная р.l(50)')]

## rag

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-4o-mini")

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
Ты – эксперт по подбору и производству спецодежды. 
Ты получаешь на вход следующие JSON-структуры, которые надо дополнить недостоющими параметрами(статусы, аналоги):
{question}
1. **Общая характеристика заявки**:
    ```json
    [
        {{
            "номер заявки": "UUID",    // Генерируй уникальный UUID v4
            "дата": "datetime",       // Дата поступления заявки
            "товары": [int]           // Список уникальных ID товаров в этой заявке
            "статус": string            // Определяется по правилам ниже
        }}
    ]
    ```

2. **Детализация каждого товара в заявке**:
    ```json
    [
        {{
            "ID": int,                    // ID товара из первого JSON
            "описание": "string",         // Описание товара (если нет, оставь пустым)
            "наименование": "string",     // Наименование товара
            "размеры": ["string/int"],    // Доступные размеры (если нет, укажи [])
            "кол-во": ["int"],            // Количество по размерам или общее (если размеров нет, укажи [общее число])
            "статус": "string",           // Определяется по правилам ниже
            "аналоги": ["string"]         // Перечисли найденные аналоги (если нет аналогов, укажи [])
        }}
    ]
    ```

### Логика определения **"статуса"** для товаров:
- `"ШИТЬ"` → Если товар является спецодеждой.
- `"КОМПЛЕМЕНТАРНОЕ"` → Если товар не является одеждой (например, ботинки, кепки, очки).
- `"НЕРЕЛЕВАНТНО"` → Если товар не относится к спецодежде и СИЗ.

### Заполнение поля **"аналоги"**:
- Подбирай аналоги товара из `{retrieved_products}`.
- Если есть подходящие аналоги, укажи их наименования в массиве.
- Если аналогов нет, укажи `[]`.

### Логика определения **"статуса"** заявки:
- `"ШИТЬ"` → Если хотя бы один товар в JSON 2 имеет статус `"ШИТЬ"`.
- `"АНАЛОГИ"` → Если у всех товаров со статусом `"ШИТЬ"` есть аналоги.
- `"НЕРЕЛЕВАНТНО"` → Если хотя бы один товар имеет статус `"НЕРЕЛЕВАНТНО"`.

### Формат ответа:
Ответ должен содержать **два массива**, разделенных словом `next`, не пиши ```json```:


[ ... ]  // JSON 1
next
[ ... ]  // JSON 2
"""

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(input_variables=["retrieved_products", "question"], template=template)

llm_chain = prompt | llm | StrOutputParser()

In [25]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

rag_chain = {"retrieved_products":retriever , "question": RunnablePassthrough()} | llm_chain

In [26]:
request, products = rag_chain.invoke(json_data).split("next")

In [30]:
df_zayavki = pd.DataFrame(json.loads(request))
df_tovary = pd.DataFrame(json.loads(products))

In [31]:
df_zayavki

,номер заявки,дата,товары,статус
0,b2c7a74e-9449-4f2c-9b2f-d39a3e14b06f,2023-10-01T00:00:00,"[1, 2, 3]",ШИТЬ


In [32]:
df_tovary

,ID,описание,наименование,размеры,кол-во,статус,аналоги
0,1,,Кепки,[one-size],[100],КОМПЛЕМЕНТАРНОЕ,[]
1,2,,Футболки с логотипом на груди,"[48-50, 52-54, 60-62]","[2, 1, 5]",ШИТЬ,[]
2,3,,Футболки Поло с двумя логотипами,"[48-50, 52-54, 56-58]","[2, 3, 2]",ШИТЬ,[]


## выявление основных характеристик товаров с помощью llm 

In [2]:
from openai import OpenAI
import json
import random
import numpy as np
import os

In [39]:
def load_product_descriptions(json_files):
    descriptions = []
    for file in json_files:
        file = f"datasets_json/json_analysis/{file}"
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            descriptions.extend(item["name"] + item['description'] for item in data)
    random.shuffle(descriptions)
    return descriptions

def analyze_descriptions_with_llm(descriptions):
    prompt = f"""
    У тебя есть список описаний товаров. 
    Выдели общие параметры, которые характеризуют эти товары, и перечисли возможные значения для каждого параметра.
    По этим параметрам будет производиться поиск товаров. 
    Выдели максимальное возможное осмысленное количество параметров. 
    Ответ представь в формате JSON, где ключ - это название параметра, а значение - список возможных значений.
    
    Описания товаров:
    {descriptions}  # Ограничение первых 5 для краткости
    """
    client = OpenAI()

    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Ты - аналитик товарных данных."},
        {"role": "user", "content": prompt}
  ]
)
    
    return completion.choices[0].message.content


In [40]:
json_files = os.listdir("datasets_json/json_analysis")
descriptions = load_product_descriptions(json_files)
num = len(descriptions)

In [41]:
num_des = np.arange(0, num, 100)

In [42]:
num_des

array([  0, 100, 200, 300, 400, 500, 600])

In [43]:
for i, p in enumerate(num_des):
    result = analyze_descriptions_with_llm(descriptions[p:p+100])
    with open(f"datasets_json/analysis_params/params_{i}.json", "w", encoding="utf-8") as f:
        f.write(result)

In [9]:
for file_name in os.listdir("datasets_json/analysis_params"):
    file = "datasets_json/analysis_params/" + file_name
    with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            params = data.keys()
            print(list(params))

['Категория', 'Тип изделия', 'Материал', 'Цвет', 'Размер', 'Сезон', 'Назначение', 'Вес', 'Рост', 'Страна производитель']
['Тип товара', 'Назначение/Функция', 'Бренд/Название', 'Цвет', 'Размер', 'Материал', 'Особенности', 'Объем/Вес', 'Срок действия/годности', 'Количество в упаковке']
['Тип изделия', 'Материал', 'Цвет', 'Размер', 'Назначение', 'Особенности', 'Класс защиты', 'Объем']
['Категория', 'Тип товара', 'Материал', 'Цвет', 'Размер/Рост', 'Защитные свойства', 'Тип застежки', 'Количество в упаковке']
['category', 'product_type', 'material', 'color', 'volume', 'size', 'quantity', 'for', 'brand']
['category', 'product_type', 'material', 'color', 'capacity', 'size', 'protection_class', 'closure_type', 'expiry_date', 'features', 'application']
['Тип изделия', 'Назначение', 'Материал', 'Цвет', 'Размер', 'Количество', 'Вес', 'Объем', 'Стандарт', 'Специальные свойства']


### Общие параметры (совпадающие по смыслу):  
- Категория / category  
- Тип изделия / Тип товара / product_type  
- Материал / material  
- Цвет / color  
- Размер / size  
- Назначение / for / application  
- Объем / volume / capacity  
- Вес / Объем/Вес  
- Количество / quantity / Количество в упаковке  
- Особенности / features  

### Различающиеся параметры:  
- Сезон  
- Страна производитель  
- Бренд / Название  
- Срок действия / годности / expiry_date  
- Рост / Размер/Рост  
- Класс защиты / protection_class  
- Защитные свойства  
- Тип застежки / closure_type  
- Стандарт  
- Специальные свойства

In [15]:
from pprint import pprint
#search = ["Особенности", "features"]
#search = ["Категория", "category"]
#search = ["Материал", "material"]
#search = ["Сезон"]
#search = ["Тип изделия", "Тип товара", "product_type"]
#search = ["Назначение", "for", "application"]
#search = ["Страна производитель"]
#search = ["Бренд", "Название"]
#search = ["Класс защиты", "protection_class"]
search = ["Защитные свойства"]
#search = ["Тип застежки", "closure_type"]
#search = ["Стандарт"]
#search = ["Специальные свойства"]

for file_name in os.listdir("datasets_json/analysis_params"):
    file = "datasets_json/analysis_params/" + file_name
    with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for i in range(len(search)):
                if search[i] in data:
                    pprint(data[search[i]])

['Водонепроницаемый',
 'Защита от механических воздействий',
 'Защита от теплового излучения',
 'Защита от химических веществ',
 'Светоотражающий',
 'Антипрокольный',
 'Гипоаллергенный']


**Вывод**: большинство специфических характеристик были выявлены в одном случае из 7ми. Это свидетельствует о том, что модели сложно проанализировать товар по скудному описанию и "додумать" характеристики.

## Ensemble Retriever: vectore and BM25

In [48]:
from langchain_core.documents import Document


docs = []

for file_name in os.listdir("datasets_json/json_price_full"):
    file_path = "datasets_json/json_price_full/" + file_name
    with open(file_path, "r", encoding="utf-8") as f:
        products = json.load(f)

    for product in products:
        metadata = product["metadata"]

        # Create a LangChain Document object
        doc = Document(page_content=product["text"], metadata=metadata)
        docs.append(doc)

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

bm25_retriever = BM25Retriever.from_documents(documents=docs)
#bm25_retriever.k = 2

In [65]:
bm25_retriever.get_relevant_documents('курка, полукомбинезон ')

[Document(metadata={'category': 'Костюм', 'season': 'Демисезон', 'gender': 'Унисекс', 'material': 'Канвас, 100% хлопок', 'sizes': '44-46/5, 52-54/2, 56-58/2, 56-58/3', 'stock': '2, 3, 5, 1', 'price': 4700, 'currency': 'RUB'}, page_content='костюм строймастер (куртка/пк) канваскомплектность: курка, полукомбинезон цвет: синий / чёрный ткань: канвас 100% хлопок, 480 гр/м2.'),
 Document(metadata={'category': 'Костюм', 'season': 'Демисезон', 'gender': 'Унисекс', 'material': '35% хлопок, 65% полиэфир', 'sizes': '44-46/2, 44-46/4, 44-46/5, 48-50/2, 48-50/4, 48-50/5, 52-54/2, 52-54/3, 52-54/4, 52-54/5, 56-58/2, 56-58/3, 56-58/4, 56-58/5, 60-62/2, 60-62/3, 60-62/4, 60-62/5, 64-66/2, 64-66/3, 64-66/4', 'stock': '8, 13, 10, 20, 14, 4, 15, 14, 18, 6, 17, 6, 7, 7, 15, 18, 3, 7, 5, 10, 9', 'price': 4200, 'currency': 'RUB'}, page_content='костюм флаер (куртка/п/к)комплектность: курка, полукомбинезон цвет: чёрный / красный ткань ткань "фаворит-2" (35% хлопок, 65% полиэфир) 240гр/м2.'),
 Document(metad

In [ ]:
from langchain_chroma import Chroma

uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store = Chroma(embedding_function = embedding, collection_name = "products", persist_directory="./chroma_db_price_full")

vector_store.add_documents(documents=docs, ids=uuids)


In [59]:
chroma_retriever = vector_store.as_retriever()

In [63]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever],
                                       weights=[0.8, 0.2])

In [64]:
ensemble_retriever.get_relevant_documents("флаер костюм")

[Document(metadata={'category': 'Костюм', 'season': 'Демисезон', 'gender': 'Унисекс', 'material': '35% хлопок, 65% полиэфир', 'sizes': '44-46/3, 48-50/3, 52-54/4', 'stock': '22, 1, 7', 'price': 2300, 'currency': 'RUB'}, page_content='костюм флаер (куртка/брюки)комплектность: курка, брюки цвет: чёрный / красный ткань ткань "фаворит-2" (35% хлопок, 65% полиэфир) 240гр/м2.'),
 Document(metadata={'category': 'Костюм', 'season': 'Демисезон', 'gender': 'Унисекс', 'material': '35% хлопок, 65% полиэфир', 'sizes': '44-46/2, 44-46/4, 44-46/5, 48-50/2, 48-50/4, 48-50/5, 52-54/2, 52-54/3, 52-54/4, 52-54/5, 56-58/2, 56-58/3, 56-58/4, 56-58/5, 60-62/2, 60-62/3, 60-62/4, 60-62/5, 64-66/2, 64-66/3, 64-66/4', 'stock': '8, 13, 10, 20, 14, 4, 15, 14, 18, 6, 17, 6, 7, 7, 15, 18, 3, 7, 5, 10, 9', 'price': 4200, 'currency': 'RUB'}, page_content='костюм флаер (куртка/п/к)комплектность: курка, полукомбинезон цвет: чёрный / красный ткань ткань "фаворит-2" (35% хлопок, 65% полиэфир) 240гр/м2.'),
 Document(metad

## BM25 and ElasticSearch

### creating db

In [39]:
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = ElasticsearchStore(
    "elastic-price-bd", embedding=embeddings, es_url="http://localhost:9200"
)

In [40]:
from uuid import uuid4
from langchain_core.documents import Document


docs = []

for file_name in os.listdir("datasets_json/json_price_full"):
    file_path = "datasets_json/json_price_full/" + file_name
    with open(file_path, "r", encoding="utf-8") as f:
        products = json.load(f)

    for product in products:
        metadata = product["metadata"]

        # Create a LangChain Document object
        doc = Document(page_content=product["text"], metadata=metadata)
        docs.append(doc)

In [41]:
uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs, ids=uuids)

['10a658e0-c6f3-40cd-9f64-a7ee1e1e203d',
 'e714d123-d606-4ca3-8592-e4acad994828',
 '20d79934-aa1b-4c31-b349-5880be980f66',
 '3ab08c21-8a4a-4f6b-8658-eed8a582e15a',
 'a0106040-7753-4410-b909-d8ae08f180c4',
 '5befdf10-03b0-4994-9ab8-edc727d95ed7',
 'e5b60562-0622-4b6e-a5cb-cbf2f438b4ec',
 'f0ce3a6a-e750-4241-bac0-7237cc5640b1',
 '681dd3c5-6de9-483c-b361-0d3705284f6f',
 'bdffd922-ac6e-4338-83f3-c4a4ad08cba1',
 '504700b6-728e-4c28-a041-8b46a98119fa',
 'bef9694d-498d-498e-83a1-5a8a02ea0c59',
 '51ec0acc-ba88-4d91-b7e1-4166f1526588',
 'b7094ddd-bca2-43a1-b99c-7772b43f50ed',
 'c3b37cdd-d524-42db-8928-68434dd8a3c5',
 'ac2711a6-988f-47db-8082-9a6f9c71c21d',
 '78009f87-3677-430a-bbab-2619abf7593c',
 'd785e2d0-702f-4471-86f0-cc1655ecc4c2',
 'dc2ea74d-15fc-4c99-9e05-5ec1a29fca1e',
 'f9271855-4e98-4525-897d-e55331c2b256',
 '6a9f8fea-b330-400d-8768-27e37110137f',
 '0e42e89e-0a97-4850-8d63-ef409c87c8f4',
 'e8ca7ddd-d230-4eb8-990d-22316bf7f88f',
 'fb5a45a9-e67d-4cfd-a0e9-6c26346916a1',
 '3e0209ac-7caa-

### using db

In [45]:
from langchain.embeddings import OpenAIEmbeddings
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy


# Initialize Elasticsearch connection
es = Elasticsearch("http://localhost:9200")  # Update with your ES instance

# Define index name
index_name = "elastic-price-bd"

# Load embeddings model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize ElasticsearchStore for vector search
vector_store = ElasticsearchStore.from_documents(
    documents=docs,
    index_name=index_name,
    embedding=embedding_model,
    es_connection=es,
    strategy=DenseVectorStrategy(hybrid=True)
)

In [ ]:
retriever = vector_store.as_retriever()

In [47]:
retriever.invoke("кепка")

BadRequestError: BadRequestError(400, 'parsing_exception', 'Unknown key for a START_OBJECT in [retriever].')